In [1]:
from scipy.io import wavfile
import numpy as np

In [2]:
import pyworld as pw

In [3]:
WAV_FILE = "./download.wav"

In [4]:
fs, data = wavfile.read(WAV_FILE)

In [5]:
data = data.astype(np.float)

In [6]:
data

array([-483., -463., -436., ...,  426.,  412.,  406.])

In [7]:
#基本周波数の抽出
_f0, t = pw.dio(data, fs)

In [8]:
#基本周波数の修正
f0 = pw.stonemask(data, _f0, t, fs)

In [9]:
#スペクトル包絡の抽出
sp = pw.cheaptrick(data, f0, t, fs)

In [10]:
#非周期性指標の抽出
ap = pw.d4c(data, f0, t, fs)

In [11]:
synthesized = pw.synthesize(f0, sp, ap, fs)

In [12]:
import IPython.display as display

In [13]:
print('original')
display.display(
    display.Audio(data, rate=fs)
)

original


In [14]:
print('synthesized')
display.display(
    display.Audio(synthesized, rate=fs)
)

synthesized


In [15]:
high_freq = pw.synthesize(f0*2.0, sp, ap, fs)
print('high_freq')
display.display(
    display.Audio(high_freq, rate=fs)
)

high_freq


In [16]:
robot_like_f0 = np.ones_like(f0)*100
robot_like = pw.synthesize(robot_like_f0, sp, ap, fs)
print('robot_like')
display.display(
    display.Audio(robot_like, rate=fs)
)

robot_like


In [17]:
female_like_sp = np.zeros_like(sp)
for f in range(female_like_sp.shape[1]):
    female_like_sp[:,f] = sp[:, int(f/1.2)]
female_like = pw.synthesize(f0*2, female_like_sp, ap, fs)
print("female_like")
display.display(
    display.Audio(female_like, rate=fs)
)

female_like


In [18]:
#mpm algorithm

In [19]:
import scipy.signal as sig

In [20]:
import numpy as np
!pip install pyfftw
from pyfftw.interfaces.numpy_fft import fft, ifft

In [21]:
#自己相関関数
def autocorrelation(sig):
    len_sig = len(sig)
    sig = np.pad(sig, (0, len_sig), "constant")
    spec = fft(sig)
    return ifft(spec * spec.conj()).real[:len_sig]

In [22]:
#NSDFというものを求める
def normalized_square_difference(sig):
    corr = autocorrelation(sig)
    N = len(sig)
    m = np.zeros([N])
    m[len(sig)-1] = sig[N-1]*sig[N-1]
    for i in range(0, len(sig)-1, -1):
        m[i] = m[i+1] + sig[i]*sig[i]
    m[m<1] = 1 #発散を防ぐ
    return corr/(corr[0] + m)

In [23]:
MPM_K = 0.5

def estimate_period(diff):
    start = 0
    while diff[start] > 0:
        start += 1
        if start >= len(diff):
            return None
    
    threshold = MPM_K * np.max(diff[start:])
    isNegative = True
    max_index = None
    for i in range(start, len(diff)):
        if isNegative:
            if diff[i] < 0:
                continue
            max_index = i
            isNegative = False
        if diff[i] < 0:
            isNegative = True
            if diff[max_index] >= threshold:
                return max_index
        if diff[i] > diff[max_index]:
            max_index = i
    return None

    '''
        for i in range(start, len(diff)):
        max_val = -1
        max_index = None
        if diff[i] < 0:
            if max_val < 0:
                continue
            else:
                return max_num
        if diff[i] > max_val:
            if diff[i] >= threshold:
                max_index = i
                max_val = diff[i]
    '''

In [24]:
def parabolic_interpolation(array, x):
    x_result = None
    if x < 1:
        x_result = x if array[x] <= array[x + 1] else x + 1
    elif x >= len(array) - 1:
        x_result = x if array[x] <= array[x - 1] else x - 1
    else:
        denom = array[x + 1] + array[x - 1] - 2 * array[x]
        delta = array[x - 1] - array[x + 1]
        if denom == 0:
            return x
        return x + delta / (2 * denom)
    return x_result

def mpm(data, samplerate):
    nsd = normalized_square_difference(data)
    index = estimate_period(nsd)
    if index is None:
        return np.nan
    return samplerate / parabolic_interpolation(nsd, index)